In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys, os
sys.path.append(os.path.abspath("../.."))
from configs import GOOGLE_APPLICATION_CREDENTIALS,GCS_BUCKET_NAME,GCS_PROJECT_ID
from google.cloud import bigquery
from src.utils.io_utils import upload_to_bigquery
from clean_utils import *

In [2]:
client = bigquery.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
table_id = f"{GCS_PROJECT_ID}.{GCS_BUCKET_NAME}.data_cleaned"

In [3]:
query = """SELECT *
FROM `khangtestdbt.xecupredict.bonbanh_com` """
data_bonbanh = client.query(query).to_dataframe()
data_bonbanh.head(1)

,Name,Năm_sản_xuất,Tình_trạng,Số_Km_đã_đi,Xuất_xứ,Kiểu_dáng,Động_cơ,Màu_ngoại_thất,Màu_nội_thất
0,Xe \n\t\t\t\n\t\t\t\t\t\t\tVinFast VF3 \n\t\t\...,2025,Xe mới,None,Lắp ráp trong nước,SUV,Điện,Hồng,Đen


In [4]:
df_bonbanh = data_bonbanh.copy()

In [5]:
df_bonbanh.columns

Index(['Name', 'Năm_sản_xuất', 'Tình_trạng', 'Số_Km_đã_đi', 'Xuất_xứ',
       'Kiểu_dáng', 'Động_cơ', 'Màu_ngoại_thất', 'Màu_nội_thất'],
      dtype='object')

In [6]:
df_bonbanh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Name            3000 non-null   object
 1   Năm_sản_xuất    3000 non-null   object
 2   Tình_trạng      3000 non-null   object
 3   Số_Km_đã_đi     2433 non-null   object
 4   Xuất_xứ         3000 non-null   object
 5   Kiểu_dáng       3000 non-null   object
 6   Động_cơ         3000 non-null   object
 7   Màu_ngoại_thất  3000 non-null   object
 8   Màu_nội_thất    3000 non-null   object
dtypes: object(9)
memory usage: 211.1+ KB


In [7]:
df_bonbanh.isna().sum()

Name                0
Năm_sản_xuất        0
Tình_trạng          0
Số_Km_đã_đi       567
Xuất_xứ             0
Kiểu_dáng           0
Động_cơ             0
Màu_ngoại_thất      0
Màu_nội_thất        0
dtype: int64

In [8]:
df_bonbanh.describe(include='all').T

,count,unique,top,freq
Name,3000,2615,Xe \n\t\t\t\n\t\t\t\t\t\t\tToyota Land Cruiser...,24
Năm_sản_xuất,3000,25,2025,596
Tình_trạng,3000,2,Xe đã dùng,2433
Số_Km_đã_đi,2433,322,"40,000 Km",123
Xuất_xứ,3000,2,Lắp ráp trong nước,1527
Kiểu_dáng,3000,9,SUV,1455
Động_cơ,3000,68,Xăng 2.0 L,753
Màu_ngoại_thất,3000,17,Trắng,1077
Màu_nội_thất,3000,18,Đen,1408


1.name

In [9]:
df_bonbanh["name"] = df_bonbanh["Name"].apply(clean_name)
df_bonbanh.drop(columns=["Name"], inplace=True)
df_bonbanh.head(1)

,Năm_sản_xuất,Tình_trạng,Số_Km_đã_đi,Xuất_xứ,Kiểu_dáng,Động_cơ,Màu_ngoại_thất,Màu_nội_thất,name
0,2025,Xe mới,None,Lắp ráp trong nước,SUV,Điện,Hồng,Đen,VinFast VF3 2025 275 Triệu


2.price

In [10]:
def extract_price(name_str):
    if not isinstance(name_str, str):
        return None

    name_str = name_str.strip()
    
    match = re.search(r'(\d+(\.\d+)?\s*t[ỷi](?:\s*\d+\s*triệu)?)|(\d+(\.\d+)?\s*triệu)', name_str, re.IGNORECASE)
    if match:
        return match.group(0)
    return None

In [11]:
df_bonbanh['price'] = df_bonbanh['name'].apply(extract_price)
df_bonbanh.head(5)

,Năm_sản_xuất,Tình_trạng,Số_Km_đã_đi,Xuất_xứ,Kiểu_dáng,Động_cơ,Màu_ngoại_thất,Màu_nội_thất,name,price
0,2025,Xe mới,None,Lắp ráp trong nước,SUV,Điện,Hồng,Đen,VinFast VF3 2025 275 Triệu,275 Triệu
1,2024,Xe đã dùng,"22,000 Km",Lắp ráp trong nước,Hatchback,Điện,Xám,Nhiều màu,VinFast VF3 Plus 2024 225 Triệu,225 Triệu
2,2024,Xe đã dùng,"10,012 Km",Lắp ráp trong nước,Hatchback,Điện,Xám,Nhiều màu,VinFast VF3 Plus 2024 222 Triệu,222 Triệu
3,2024,Xe đã dùng,"10,000 Km",Lắp ráp trong nước,Hatchback,Điện,Vàng,Nhiều màu,VinFast VF3 Plus 2024 225 Triệu,225 Triệu
4,2025,Xe mới,None,Lắp ráp trong nước,Hatchback,Điện,Xanh,Đen,VinFast VF3 Plus 2025 289 Triệu,289 Triệu


In [12]:
def clean_price(x):
    if pd.isna(x):
        return None
    
    x = str(x).lower().strip()

    if x in ["", "none", "nan", "giá liên hệ", "contact", "-"]:
        return None

    x = x.replace("triệu", "triệu").replace("trieuj", "triệu")
    x = x.replace("ty", "tỷ").replace("ti", "tỷ")

    x = x.replace("tỉ", "tỷ")

    x = re.sub(r"\s+", " ", x)

    if "triệu" not in x and "tỷ" not in x:
        digits = re.sub(r"[^\d]", "", x)
        return int(digits) if digits.isdigit() else None

    x = x.replace(",", ".")  

    match = re.match(r"(\d*\.?\d*)\s*tỷ\s*(\d*\.?\d*)?\s*triệu?", x)
    if match:
        ty = float(match.group(1)) if match.group(1) else 0
        trieu = float(match.group(2)) if match.group(2) else 0
        return int(ty * 1_000_000_000 + trieu * 1_000_000)

    if "tỷ" in x:
        num = re.findall(r"[\d\.]+", x)
        if num:
            return int(float(num[0]) * 1_000_000_000)
        return None

    if "triệu" in x:
        num = re.findall(r"[\d\.]+", x)
        if num:
            return int(float(num[0]) * 1_000_000)
        return None

    return None


In [13]:
df_bonbanh["price"] = df_bonbanh["price"].apply(clean_price)
df_bonbanh.sample(5)

,Năm_sản_xuất,Tình_trạng,Số_Km_đã_đi,Xuất_xứ,Kiểu_dáng,Động_cơ,Màu_ngoại_thất,Màu_nội_thất,name,price
1189,2014,Xe đã dùng,"86,000 Km",Nhập khẩu,SUV,Xăng 5.0 L,Đen,Nâu,LandRover Range Rover Autobiography 5 0 2014 2...,2385000000
2146,2021,Xe đã dùng,"50,000 Km",Nhập khẩu,SUV,Xăng 1.8 L,Xanh,Đỏ,Toyota Corolla Cross 1 8V 2021 650 Triệu,650000000
671,2016,Xe đã dùng,"74,000 Km",Lắp ráp trong nước,Sedan,Xăng 4.7 L,Trắng,Nâu,Mercedes Benz S class S500L 2016 1 Tỷ 520 Triệu,1520000000
1597,2023,Xe đã dùng,"45,000 Km",Nhập khẩu,Bán tải / Pickup,Dầu 1.9 L,Bạc,Đen,Isuzu Dmax Prestige 1 9L 4x2 MT 2023 499 Triệu,499000000
139,2023,Xe đã dùng,"30,000 Km",Lắp ráp trong nước,SUV,Điện,Đen,Kem,VinFast VF8 Plus AWD 2023 765 Triệu,765000000


In [14]:
df_bonbanh["price"].isna().sum()

np.int64(0)

In [15]:
df_bonbanh = df_bonbanh[df_bonbanh["price"] > 50000000]

3.brand

In [16]:
df_bonbanh["brand"] = df_bonbanh["name"].str.strip().str.split().str[0]
df_bonbanh["brand"] = df_bonbanh["brand"].str.lower()

In [17]:
df_bonbanh["brand"].unique()

array(['vinfast', 'volkswagen', 'volvo', 'toyota', 'mazda', 'mercedes',
       'lexus', 'lynk', 'mg', 'isuzu', 'jaguar', 'jeep', 'kia',
       'landrover', 'lamborghini', 'hyundai', 'infiniti', 'mitsubishi',
       'mini', 'peugeot', 'porsche', 'ram', 'rolls', 'smart', 'ssangyong',
       'subaru', 'suzuki', 'morgan', 'nissan', 'omoda', 'ford', 'gac',
       'gaz', 'haval', 'honda', 'bmw', 'chevrolet', 'citroen', 'daewoo',
       'daihatsu', 'dodge', 'dongfeng', 'ferrari', 'byd', 'cadillac',
       'chenglong', 'audi', 'bentley', 'bestune'], dtype=object)

4.year

In [18]:
df_bonbanh["Năm_sản_xuất"].sample(5)

1713    2021
1879    2020
372     2021
2089    2022
2399    2018
Name: Năm_sản_xuất, dtype: object

In [19]:
df_bonbanh["Năm_sản_xuất"].isna().sum()

np.int64(0)

In [20]:
df_bonbanh["age"] = 2025 - df_bonbanh["Năm_sản_xuất"].astype(int)
df_bonbanh.drop(columns=["Năm_sản_xuất"], inplace=True)
df_bonbanh["age"].sample(5)

358     0
1069    2
930     7
2566    1
1067    2
Name: age, dtype: int64

5.status

In [21]:
df_bonbanh["Tình_trạng"].unique()

array(['Xe mới', 'Xe đã dùng'], dtype=object)

In [22]:
df_bonbanh = df_bonbanh[df_bonbanh["Tình_trạng"] == "Xe đã dùng"]
df_bonbanh.drop(columns=["Tình_trạng"], inplace=True)

6.km

In [23]:
df_bonbanh["Số_Km_đã_đi"].sample(5)

626     145,000 Km
1487     42,500 Km
2768     83,000 Km
1024    100,000 Km
2562     26,000 Km
Name: Số_Km_đã_đi, dtype: object

In [24]:
df_bonbanh["Số_Km_đã_đi"].isna().sum()

np.int64(0)

In [25]:
df_bonbanh.rename(columns={"Số_Km_đã_đi": "km"}, inplace=True)

In [26]:
def clean_km(x):
    if pd.isna(x):
        return None
    
    x = str(x).lower().strip()

    x = x.replace("km", "").replace(",", "").strip()

    if x in ["", "none", "nan", "-"]:
        return None

    digits = re.sub(r"[^\d]", "", x)
    return int(digits) if digits.isdigit() else None

In [27]:
df_bonbanh["km"] = df_bonbanh["km"].apply(clean_km)

In [28]:
df_bonbanh["km"].sample(5)

1119     45000
2821     14000
500     112000
2465      9979
1117         0
Name: km, dtype: int64

In [29]:
df_bonbanh = df_bonbanh[df_bonbanh["km"] > 100]

In [30]:
df_bonbanh["km"].min()

np.int64(120)

7.origin

In [31]:
df_bonbanh["Xuất_xứ"].unique()

array(['Lắp ráp trong nước', 'Nhập khẩu'], dtype=object)

In [32]:
df_bonbanh = df_bonbanh.rename(columns={"Xuất_xứ": "origin"})

In [33]:
df_bonbanh["origin"] = df_bonbanh["origin"].replace({
    "Lắp ráp trong nước": "trong nước",
    "Nhập khẩu": "nhập khẩu"
})


In [34]:
df_bonbanh["origin"].isna().sum()

np.int64(0)

In [35]:
df_bonbanh["origin"].value_counts()

origin
trong nước    1199
nhập khẩu     1083
Name: count, dtype: int64

8.body

In [36]:
df_bonbanh["Kiểu_dáng"].unique()

array(['Hatchback', 'SUV', 'Sedan', 'Crossover', 'Van/Minivan',
       'Bán tải / Pickup', 'Coupe', 'Truck', 'Convertible/Cabriolet'],
      dtype=object)

In [37]:
df_bonbanh = df_bonbanh.rename(columns={"Kiểu_dáng": "body"})
df_bonbanh["body"] = df_bonbanh["body"].str.lower().str.strip()

In [38]:
df_bonbanh["body"].isna().sum()

np.int64(0)

In [39]:
df_bonbanh["body"].value_counts()

body
suv                      1069
sedan                     640
crossover                 190
van/minivan               127
hatchback                 125
bán tải / pickup         92
convertible/cabriolet      20
coupe                      17
truck                       2
Name: count, dtype: int64

9.fuel

In [40]:
df_bonbanh["Động_cơ"].unique()

array(['Điện', 'Xăng  2.0 L', 'Điện  0.6 L', 'Điện  0.2 L',
       'Hybrid  2.0 L', 'Xăng  1.5 L', 'Xăng  1.2 L', 'Xăng  1.3 L',
       'Xăng  1.8 L', 'Xăng  1.4 L', 'Xăng  1.6 L', 'Xăng  2.5 L',
       'Xăng  2.4 L', 'Hybrid  2.5 L', 'Xăng  3.5 L', 'Hybrid  3.5 L',
       'Dầu  2.2 L', 'Hybrid  1.5 L', 'Xăng  3.0 L', 'Xăng  4.7 L',
       'Xăng  5.5 L', 'Dầu  3.0 L', 'Dầu  2.1 L', 'Hybrid  3.0 L',
       'Điện  0.4 L', 'Xăng  4.0 L', 'Xăng  6.0 L', 'Dầu  2.0 L',
       'Xăng  5.0 L', 'Xăng', 'Hybrid  1.6 L', 'Xăng  6.5 L',
       'Xăng  5.2 L', 'Hybrid  1.8 L', 'Xăng  4.6 L', 'Hybrid  2.4 L',
       'Xăng  1.25 L', 'Xăng  1.0 L', 'Xăng  3.3 L', 'Dầu  2.5 L',
       'Xăng  3.4 L', 'Hybrid  5.0 L', 'Xăng  5.7 L', 'Dầu',
       'Xăng  3.7 L', 'Dầu  1.9 L', 'Xăng  2.7 L', 'Dầu  2.8 L',
       'Dầu  2.4 L', '-', 'Xăng  3.6 L', 'Xăng  4.8 L', 'Xăng  2.9 L',
       'Xăng  6.6 L', 'Xăng  6.7 L', 'Xăng  3.2 L', 'Dầu  2.7 L',
       'Xăng  4.5 L', 'Dầu  2.3 L', 'Dầu  3.2 L', 'Xăng  6.2 L',
    

In [41]:
df_bonbanh = df_bonbanh.rename(columns={"Động_cơ": "fuel"})
df_bonbanh["fuel"] = df_bonbanh["fuel"].apply(clean_fuel)

In [42]:
df_bonbanh["fuel"].isna().sum()

np.int64(0)

In [43]:
df_bonbanh["fuel"].unique()

array(['Điện', 'Xăng', 'Hybrid', 'Dầu', 'Khác'], dtype=object)

In [44]:
df_bonbanh["fuel"].value_counts()

fuel
Xăng      1815
Dầu        313
Hybrid      97
Điện        56
Khác         1
Name: count, dtype: int64

In [45]:
df_bonbanh = df_bonbanh[df_bonbanh["fuel"] != "Khác"]

In [46]:
df_bonbanh["fuel"].unique()

array(['Điện', 'Xăng', 'Hybrid', 'Dầu'], dtype=object)

10.dropcol

In [47]:
df_bonbanh.drop(columns=["Màu_ngoại_thất", "Màu_nội_thất"], inplace=True)

In [48]:
df_bonbanh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2281 entries, 1 to 2999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   km      2281 non-null   int64 
 1   origin  2281 non-null   object
 2   body    2281 non-null   object
 3   fuel    2281 non-null   object
 4   name    2281 non-null   object
 5   price   2281 non-null   int64 
 6   brand   2281 non-null   object
 7   age     2281 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 160.4+ KB


11.updata

In [49]:
df_bonbanh.sample(10)

,km,origin,body,fuel,name,price,brand,age
1033,51000,trong nước,suv,Xăng,Mercedes Benz GLC 200 4Matic 2021 1 Tỷ 299 Triệu,1299000000,mercedes,4
975,7000,trong nước,sedan,Hybrid,Mercedes Benz E class E200 Exclusive 2022 1 Tỷ...,1699000000,mercedes,3
1690,55500,trong nước,crossover,Xăng,Toyota Innova E 2 0 MT 2024 665 Triệu,665000000,toyota,1
832,22000,nhập khẩu,sedan,Xăng,Mercedes Benz Maybach S450 4Matic 2022 6 Tỷ 85...,6850000000,mercedes,3
2465,9979,trong nước,suv,Xăng,BMW X3 sDrive20i M Sport 2024 1 Tỷ 939 Triệu,1939000000,bmw,1
499,52000,trong nước,suv,Xăng,Mazda CX5 Premium 2 0 AT 2022 710 Triệu,710000000,mazda,3
1112,111000,nhập khẩu,suv,Xăng,LandRover Range Rover HSE 3 0 2014 1 Tỷ 950 Triệu,1950000000,landrover,11
2877,138000,nhập khẩu,suv,Xăng,Audi Q7 3 0 AT 2014 599 Triệu,599000000,audi,11
1309,70000,trong nước,suv,Xăng,Hyundai SantaFe Premium 2 4L HTRAC 2019 750 Triệu,750000000,hyundai,6
926,80000,trong nước,suv,Xăng,Mercedes Benz GLC 300 4Matic 2017 950 Triệu,950000000,mercedes,8


In [50]:
upload_to_bigquery(df_bonbanh, table_id=table_id, if_exists="replace" )

✅ Uploaded 2281 rows to khangtestdbt.xecupredict.data_cleaned


In [51]:
# end